# Tmin


In [79]:
import ee
import time
import sys
import numpy as np
import pandas as pd
import itertools
import os
import urllib
import datetime
ee.Initialize()

def cloudReduce(collection,sdate,edate,delta):
    
    temp_sdate = datetime.datetime.strptime(sdate,'%Y-%m-%d')
    real_edate = min([datetime.datetime.now(),datetime.datetime.strptime(edate,'%Y-%m-%d')])
    yearEnd_edate = min([datetime.datetime.now(),datetime.datetime.strptime(edate,'%Y-%m-%d')])
    X = []
    dateLims = []

    while temp_sdate < real_edate: #go through dates -> get images
        print ("while",temp_sdate)
        ee_sdate = datetime.datetime.strftime(temp_sdate,'%Y-%m-%d')
        temp_edate = temp_sdate + datetime.timedelta(days=delta) #diff is in DAYS (PAY ATTENTION!)
        #ee_edate = datetime.datetime.strftime(temp_edate,'%Y-%m-%d')
        ee_edate = min([datetime.datetime.strptime(str(temp_sdate.year)+'-12-31','%Y-%m-%d'),(temp_edate)])
        print (ee_edate)
        temp = (collection.filterDate(ee_sdate,ee_edate) 
            #.filterMetadata('CLOUD_COVER','less_than',np.mean(clouder)+0.01) #remove the cloudy images -> disrupting reduction
            .filterBounds(ee.Geometry.Rectangle(-106.5, 50,-64, 23)).select(['temperature'])\
            #.select(['R_p'+spct,'G_p'+spct,'B_p'+spct,'NIR_p'+spct,'SWIR_p'+spct],['R','G','B','NIR','SWIR']))
            .reduce(ee.Reducer.min()))
            #.select(['R_mean','G_mean','B_mean','NIR_mean','SWIR_mean'],['R','G','B','NIR','SWIR']))

        X.append(temp)
        dateLims.append([temp_sdate,temp_edate])
        temp_sdate = min(temp_edate,  ee_edate+ datetime.timedelta(days=1))
        print (temp_sdate) 
    return ee.ImageCollection(X)

def export_oneimage(img,folder,name,scale,crs):
  task = ee.batch.Export.image(img, name, {
      'driveFolder':folder,
      'driveFileNamePrefix':name,
      'scale':scale,
      'crs':crs
  })
  task.start()
  while task.status()['state'] == 'RUNNING':
    print ('Running...')
    # Perhaps task.cancel() at some point.
    time.sleep(10)
  print ('Done.'), task.status()







def appendBand(current, previous):
    # Rename the band
    previous=ee.Image(previous)
    current = current.select([0])
    # Append it to the result (Note: only return current item on first element/iteration)
    accum = ee.Algorithms.If(ee.Algorithms.IsEqual(previous,None), current, previous.addBands(ee.Image(current)))
    # Return the accumulation
    return accum



# county_region = ee.FeatureCollection('ft:1S4EB6319wWW2sWQDPhDvmSBIVrD3iEmCLYB7nMM')


locations = pd.read_csv('locations_final.csv')


#return
#var image = ee.Image().float().paint(dataset, 'STATEFP');

collection = ee.ImageCollection("NASA/NLDAS/FORA0125_H002")
delta = 3
imgcoll = cloudReduce(collection,'2018-01-01','2019-01-01',delta)

img=imgcoll.iterate(appendBand)
img=ee.Image(img)

for loc1, loc2, lat, lon in locations.values:
    fname = '{}_{}'.format(int(loc1), int(loc2))
    fname = '{}_{}'.format(int(loc1), int(loc2))
    print(fname)

    # offset = 0.11
    scale  = 500
    crs='EPSG:4326'

    # filter for a county
    region = county_region.filterMetadata('STATEFP', 'equals', str(int(loc1)))
    region = ee.FeatureCollection(region).filterMetadata('COUNTYFP', 'equals', str("{0:03}".format(int(loc2))))
    region = ee.Feature(region.first())
    region.getInfo()


    

    while True:
        try:
            export_oneimage(img.clip(region), '2018_Tmin_3Day', fname, scale, crs)
        except:
            print ('retry')
            time.sleep(10)
            continue
        break


DEBUG
while 2018-01-01 00:00:00
2018-01-04 00:00:00
2018-01-04 00:00:00
while 2018-01-04 00:00:00
2018-01-07 00:00:00
2018-01-07 00:00:00
while 2018-01-07 00:00:00
2018-01-10 00:00:00
2018-01-10 00:00:00
while 2018-01-10 00:00:00
2018-01-13 00:00:00
2018-01-13 00:00:00
while 2018-01-13 00:00:00
2018-01-16 00:00:00
2018-01-16 00:00:00
while 2018-01-16 00:00:00
2018-01-19 00:00:00
2018-01-19 00:00:00
while 2018-01-19 00:00:00
2018-01-22 00:00:00
2018-01-22 00:00:00
while 2018-01-22 00:00:00
2018-01-25 00:00:00
2018-01-25 00:00:00
while 2018-01-25 00:00:00
2018-01-28 00:00:00
2018-01-28 00:00:00
while 2018-01-28 00:00:00
2018-01-31 00:00:00
2018-01-31 00:00:00
while 2018-01-31 00:00:00
2018-02-03 00:00:00
2018-02-03 00:00:00
while 2018-02-03 00:00:00
2018-02-06 00:00:00
2018-02-06 00:00:00
while 2018-02-06 00:00:00
2018-02-09 00:00:00
2018-02-09 00:00:00
while 2018-02-09 00:00:00
2018-02-12 00:00:00
2018-02-12 00:00:00
while 2018-02-12 00:00:00
2018-02-15 00:00:00
2018-02-15 00:00:00
whil

# Tmax

In [75]:
import ee
import time
import sys
import numpy as np
import pandas as pd
import itertools
import os
import urllib
import datetime
ee.Initialize()

def cloudReduce(collection,sdate,edate,delta):
    
    temp_sdate = datetime.datetime.strptime(sdate,'%Y-%m-%d')
    real_edate = min([datetime.datetime.now(),datetime.datetime.strptime(edate,'%Y-%m-%d')])
    yearEnd_edate = min([datetime.datetime.now(),datetime.datetime.strptime(edate,'%Y-%m-%d')])
    X = []
    dateLims = []

    while temp_sdate < real_edate: #go through dates -> get images
        print ("while",temp_sdate)
        ee_sdate = datetime.datetime.strftime(temp_sdate,'%Y-%m-%d')
        temp_edate = temp_sdate + datetime.timedelta(days=delta) #diff is in DAYS (PAY ATTENTION!)
        #ee_edate = datetime.datetime.strftime(temp_edate,'%Y-%m-%d')
        ee_edate = min([datetime.datetime.strptime(str(temp_sdate.year)+'-12-31','%Y-%m-%d'),(temp_edate)])
        print (ee_edate)
        temp = (collection.filterDate(ee_sdate,ee_edate) 
            #.filterMetadata('CLOUD_COVER','less_than',np.mean(clouder)+0.01) #remove the cloudy images -> disrupting reduction
            .filterBounds(ee.Geometry.Rectangle(-106.5, 50,-64, 23)).select(['temperature'])\
            #.select(['R_p'+spct,'G_p'+spct,'B_p'+spct,'NIR_p'+spct,'SWIR_p'+spct],['R','G','B','NIR','SWIR']))
            .reduce(ee.Reducer.max()))
            #.select(['R_mean','G_mean','B_mean','NIR_mean','SWIR_mean'],['R','G','B','NIR','SWIR']))

        X.append(temp)
        dateLims.append([temp_sdate,temp_edate])
        temp_sdate = min(temp_edate,  ee_edate+ datetime.timedelta(days=1))
        print (temp_sdate) 
    return ee.ImageCollection(X)

def export_oneimage(img,folder,name,scale,crs):
  task = ee.batch.Export.image(img, name, {
      'driveFolder':folder,
      'driveFileNamePrefix':name,
      'scale':scale,
      'crs':crs
  })
  task.start()
  while task.status()['state'] == 'RUNNING':
    print ('Running...')
    # Perhaps task.cancel() at some point.
    time.sleep(10)
  print ('Done.'), task.status()




# Transforms an Image Collection with 1 band per Image into a single Image with items as bands


def appendBand(current, previous):
    # Rename the band
    previous=ee.Image(previous)
    current = current.select([0])
    # Append it to the result (Note: only return current item on first element/iteration)
    accum = ee.Algorithms.If(ee.Algorithms.IsEqual(previous,None), current, previous.addBands(ee.Image(current)))
    # Return the accumulation
    return accum



# county_region = ee.FeatureCollection('ft:1S4EB6319wWW2sWQDPhDvmSBIVrD3iEmCLYB7nMM')

print('DEBUG')
#return
county_region = ee.FeatureCollection("TIGER/2016/Counties")

locations = pd.read_csv('locations_final.csv')



collection = ee.ImageCollection("NASA/NLDAS/FORA0125_H002")
delta = 3
imgcoll = cloudReduce(collection,'2018-01-01','2019-01-01',delta)

img=imgcoll.iterate(appendBand)
img=ee.Image(img)

for loc1, loc2, lat, lon in locations.values:
    fname = '{}_{}'.format(int(loc1), int(loc2))
    fname = '{}_{}'.format(int(loc1), int(loc2))
    print(fname)

    # offset = 0.11
    scale  = 500
    crs='EPSG:4326'

    # filter for a county
    region = county_region.filterMetadata('STATEFP', 'equals', str(int(loc1)))
    region = ee.FeatureCollection(region).filterMetadata('COUNTYFP', 'equals', str("{0:03}".format(int(loc2))))
    region = ee.Feature(region.first())
    region.getInfo()



    while True:
        try:
            export_oneimage(img.clip(region), '2018_Tmax_3Day', fname, scale, crs) #Met_test_Tmax
        except:
            print ('retry')
            time.sleep(10)
            continue
        break


DEBUG
while 2018-01-01 00:00:00
2018-01-04 00:00:00
2018-01-04 00:00:00
while 2018-01-04 00:00:00
2018-01-07 00:00:00
2018-01-07 00:00:00
while 2018-01-07 00:00:00
2018-01-10 00:00:00
2018-01-10 00:00:00
while 2018-01-10 00:00:00
2018-01-13 00:00:00
2018-01-13 00:00:00
while 2018-01-13 00:00:00
2018-01-16 00:00:00
2018-01-16 00:00:00
while 2018-01-16 00:00:00
2018-01-19 00:00:00
2018-01-19 00:00:00
while 2018-01-19 00:00:00
2018-01-22 00:00:00
2018-01-22 00:00:00
while 2018-01-22 00:00:00
2018-01-25 00:00:00
2018-01-25 00:00:00
while 2018-01-25 00:00:00
2018-01-28 00:00:00
2018-01-28 00:00:00
while 2018-01-28 00:00:00
2018-01-31 00:00:00
2018-01-31 00:00:00
while 2018-01-31 00:00:00
2018-02-03 00:00:00
2018-02-03 00:00:00
while 2018-02-03 00:00:00
2018-02-06 00:00:00
2018-02-06 00:00:00
while 2018-02-06 00:00:00
2018-02-09 00:00:00
2018-02-09 00:00:00
while 2018-02-09 00:00:00
2018-02-12 00:00:00
2018-02-12 00:00:00
while 2018-02-12 00:00:00
2018-02-15 00:00:00
2018-02-15 00:00:00
whil

# Humidity

In [76]:
import ee
import time
import sys
import numpy as np
import pandas as pd
import itertools
import os
import urllib
import datetime
ee.Initialize()

def cloudReduce(collection,sdate,edate,delta):
    
    temp_sdate = datetime.datetime.strptime(sdate,'%Y-%m-%d')
    real_edate = min([datetime.datetime.now(),datetime.datetime.strptime(edate,'%Y-%m-%d')])
    yearEnd_edate = min([datetime.datetime.now(),datetime.datetime.strptime(edate,'%Y-%m-%d')])
    X = []
    dateLims = []

    while temp_sdate < real_edate: #go through dates -> get images
        print ("while",temp_sdate)
        ee_sdate = datetime.datetime.strftime(temp_sdate,'%Y-%m-%d')
        temp_edate = temp_sdate + datetime.timedelta(days=delta) #diff is in DAYS (PAY ATTENTION!)
        #ee_edate = datetime.datetime.strftime(temp_edate,'%Y-%m-%d')
        ee_edate = min([datetime.datetime.strptime(str(temp_sdate.year)+'-12-31','%Y-%m-%d'),(temp_edate)])
        print (ee_edate)
        temp = (collection.filterDate(ee_sdate,ee_edate) 
            #.filterMetadata('CLOUD_COVER','less_than',np.mean(clouder)+0.01) #remove the cloudy images -> disrupting reduction
            .filterBounds(ee.Geometry.Rectangle(-106.5, 50,-64, 23)).select(['specific_humidity'])\
            #.select(['R_p'+spct,'G_p'+spct,'B_p'+spct,'NIR_p'+spct,'SWIR_p'+spct],['R','G','B','NIR','SWIR']))
            .reduce(ee.Reducer.mean()))
            #.select(['R_mean','G_mean','B_mean','NIR_mean','SWIR_mean'],['R','G','B','NIR','SWIR']))

        X.append(temp)
        dateLims.append([temp_sdate,temp_edate])
        temp_sdate = min(temp_edate,  ee_edate+ datetime.timedelta(days=1))
        print (temp_sdate) 
    return ee.ImageCollection(X)

def export_oneimage(img,folder,name,scale,crs):
  task = ee.batch.Export.image(img, name, {
      'driveFolder':folder,
      'driveFileNamePrefix':name,
      'scale':scale,
      'crs':crs
  })
  task.start()
  while task.status()['state'] == 'RUNNING':
    print ('Running...')
    # Perhaps task.cancel() at some point.
    time.sleep(10)
  print ('Done.'), task.status()



# Transforms an Image Collection with 1 band per Image into a single Image with items as bands


def appendBand(current, previous):
    # Rename the band
    previous=ee.Image(previous)
    current = current.select([0])
    # Append it to the result (Note: only return current item on first element/iteration)
    accum = ee.Algorithms.If(ee.Algorithms.IsEqual(previous,None), current, previous.addBands(ee.Image(current)))
    # Return the accumulation
    return accum



# county_region = ee.FeatureCollection('ft:1S4EB6319wWW2sWQDPhDvmSBIVrD3iEmCLYB7nMM')


county_region = ee.FeatureCollection("TIGER/2016/Counties")

locations = pd.read_csv('locations_final_Selected_test_for_missing_county_17_39.csv')



collection = ee.ImageCollection("NASA/NLDAS/FORA0125_H002")
delta = 3
imgcoll = cloudReduce(collection,'2018-01-01','2019-01-01',delta) #'2019-05-01','2019-12-01'

img=imgcoll.iterate(appendBand)
img=ee.Image(img)

for loc1, loc2, lat, lon in locations.values:
    fname = '{}_{}'.format(int(loc1), int(loc2))
    fname = '{}_{}'.format(int(loc1), int(loc2))
    print(fname)

    # offset = 0.11
    scale  = 500
    crs='EPSG:4326'

    # filter for a county
    region = county_region.filterMetadata('STATEFP', 'equals', str(int(loc1)))
    region = ee.FeatureCollection(region).filterMetadata('COUNTYFP', 'equals', str("{0:03}".format(int(loc2))))
    region = ee.Feature(region.first())
    region.getInfo()


    

    while True:
        try:
            export_oneimage(img.clip(region), '2018_HU_3Day', fname, scale, crs)
        except:
            print ('retry')
            time.sleep(10)
            continue
        break


DEBUG
while 2018-01-01 00:00:00
2018-01-04 00:00:00
2018-01-04 00:00:00
while 2018-01-04 00:00:00
2018-01-07 00:00:00
2018-01-07 00:00:00
while 2018-01-07 00:00:00
2018-01-10 00:00:00
2018-01-10 00:00:00
while 2018-01-10 00:00:00
2018-01-13 00:00:00
2018-01-13 00:00:00
while 2018-01-13 00:00:00
2018-01-16 00:00:00
2018-01-16 00:00:00
while 2018-01-16 00:00:00
2018-01-19 00:00:00
2018-01-19 00:00:00
while 2018-01-19 00:00:00
2018-01-22 00:00:00
2018-01-22 00:00:00
while 2018-01-22 00:00:00
2018-01-25 00:00:00
2018-01-25 00:00:00
while 2018-01-25 00:00:00
2018-01-28 00:00:00
2018-01-28 00:00:00
while 2018-01-28 00:00:00
2018-01-31 00:00:00
2018-01-31 00:00:00
while 2018-01-31 00:00:00
2018-02-03 00:00:00
2018-02-03 00:00:00
while 2018-02-03 00:00:00
2018-02-06 00:00:00
2018-02-06 00:00:00
while 2018-02-06 00:00:00
2018-02-09 00:00:00
2018-02-09 00:00:00
while 2018-02-09 00:00:00
2018-02-12 00:00:00
2018-02-12 00:00:00
while 2018-02-12 00:00:00
2018-02-15 00:00:00
2018-02-15 00:00:00
whil

# NLDAS Prec 1Day

In [77]:
import ee
import time
import sys
import numpy as np
import pandas as pd
import itertools
import os
import urllib
import datetime
ee.Initialize()

def cloudReduce(collection,sdate,edate,delta):
    
    temp_sdate = datetime.datetime.strptime(sdate,'%Y-%m-%d')
    real_edate = min([datetime.datetime.now(),datetime.datetime.strptime(edate,'%Y-%m-%d')])
    yearEnd_edate = min([datetime.datetime.now(),datetime.datetime.strptime(edate,'%Y-%m-%d')])
    X = []
    dateLims = []

    while temp_sdate < real_edate: #go through dates -> get images
        print ("while",temp_sdate)
        ee_sdate = datetime.datetime.strftime(temp_sdate,'%Y-%m-%d')
        temp_edate = temp_sdate + datetime.timedelta(days=delta) #diff is in DAYS (PAY ATTENTION!)
        #ee_edate = datetime.datetime.strftime(temp_edate,'%Y-%m-%d')
        ee_edate = min([datetime.datetime.strptime(str(temp_sdate.year)+'-12-31','%Y-%m-%d'),(temp_edate)])
        print (ee_edate)
        temp = (collection.filterDate(ee_sdate,ee_edate) 
            #.filterMetadata('CLOUD_COVER','less_than',np.mean(clouder)+0.01) #remove the cloudy images -> disrupting reduction
            .filterBounds(ee.Geometry.Rectangle(-106.5, 50,-64, 23)).select(['total_precipitation'])\
            #.select(['R_p'+spct,'G_p'+spct,'B_p'+spct,'NIR_p'+spct,'SWIR_p'+spct],['R','G','B','NIR','SWIR']))
            .reduce(ee.Reducer.sum()))
            #.select(['R_mean','G_mean','B_mean','NIR_mean','SWIR_mean'],['R','G','B','NIR','SWIR']))

        X.append(temp)
        dateLims.append([temp_sdate,temp_edate])
        temp_sdate = min(temp_edate,  ee_edate+ datetime.timedelta(days=1))
        print (temp_sdate) 
    return ee.ImageCollection(X)


def export_oneimage(img,folder,name,scale,crs):
  task = ee.batch.Export.image(img, name, {
      'driveFolder':folder,
      'driveFileNamePrefix':name,
      'scale':scale,
      'crs':crs
  })
  task.start()
  while task.status()['state'] == 'RUNNING':
    print ('Running...')
    # Perhaps task.cancel() at some point.
    time.sleep(10)
  print ('Done.'), task.status()




#locations = pd.read_csv('locations_final_Selected.csv')


# Transforms an Image Collection with 1 band per Image into a single Image with items as bands
# Author: Jamie Vleeshouwer

def appendBand(current, previous):
    # Rename the band
    previous=ee.Image(previous)
    current = current.select([0])
    # Append it to the result (Note: only return current item on first element/iteration)
    accum = ee.Algorithms.If(ee.Algorithms.IsEqual(previous,None), current, previous.addBands(ee.Image(current)))
    # Return the accumulation
    return accum




county_region = ee.FeatureCollection("TIGER/2016/Counties")

locations = pd.read_csv('locations_final.csv')
#return
#var image = ee.Image().float().paint(dataset, 'STATEFP');

collection = ee.ImageCollection("NASA/NLDAS/FORA0125_H002")
delta = 3
imgcoll = cloudReduce(collection, '2018-01-01','2019-01-01',delta) #'2019-05-01','2019-12-01'

img=imgcoll.iterate(appendBand)
img=ee.Image(img)

for loc1, loc2, lat, lon in locations.values:
    fname = '{}_{}'.format(int(loc1), int(loc2))
    fname = '{}_{}'.format(int(loc1), int(loc2))
    print(fname)

    # offset = 0.11
    scale  = 500
    crs='EPSG:4326'

    # filter for a county
    region = county_region.filterMetadata('STATEFP', 'equals', str(int(loc1)))
    region = ee.FeatureCollection(region).filterMetadata('COUNTYFP', 'equals', str("{0:03}".format(int(loc2))))
    region = ee.Feature(region.first())
    region.getInfo()


    

    while True:
        try:
            export_oneimage(img.clip(region), '2018_Prec_3Day', fname, scale, crs)
        except:
            print ('retry')
            time.sleep(10)
            continue
        break


DEBUG
while 2018-01-01 00:00:00
2018-01-04 00:00:00
2018-01-04 00:00:00
while 2018-01-04 00:00:00
2018-01-07 00:00:00
2018-01-07 00:00:00
while 2018-01-07 00:00:00
2018-01-10 00:00:00
2018-01-10 00:00:00
while 2018-01-10 00:00:00
2018-01-13 00:00:00
2018-01-13 00:00:00
while 2018-01-13 00:00:00
2018-01-16 00:00:00
2018-01-16 00:00:00
while 2018-01-16 00:00:00
2018-01-19 00:00:00
2018-01-19 00:00:00
while 2018-01-19 00:00:00
2018-01-22 00:00:00
2018-01-22 00:00:00
while 2018-01-22 00:00:00
2018-01-25 00:00:00
2018-01-25 00:00:00
while 2018-01-25 00:00:00
2018-01-28 00:00:00
2018-01-28 00:00:00
while 2018-01-28 00:00:00
2018-01-31 00:00:00
2018-01-31 00:00:00
while 2018-01-31 00:00:00
2018-02-03 00:00:00
2018-02-03 00:00:00
while 2018-02-03 00:00:00
2018-02-06 00:00:00
2018-02-06 00:00:00
while 2018-02-06 00:00:00
2018-02-09 00:00:00
2018-02-09 00:00:00
while 2018-02-09 00:00:00
2018-02-12 00:00:00
2018-02-12 00:00:00
while 2018-02-12 00:00:00
2018-02-15 00:00:00
2018-02-15 00:00:00
whil

# SW Radiation

In [78]:
import ee
import time
import sys
import numpy as np
import pandas as pd
import itertools
import os
import urllib
import datetime
ee.Initialize()

def cloudReduce(collection,sdate,edate,delta):
    
    temp_sdate = datetime.datetime.strptime(sdate,'%Y-%m-%d')
    real_edate = min([datetime.datetime.now(),datetime.datetime.strptime(edate,'%Y-%m-%d')])
    yearEnd_edate = min([datetime.datetime.now(),datetime.datetime.strptime(edate,'%Y-%m-%d')])
    X = []
    dateLims = []

    while temp_sdate < real_edate: #go through dates -> get images
        print ("while",temp_sdate)
        ee_sdate = datetime.datetime.strftime(temp_sdate,'%Y-%m-%d')
        temp_edate = temp_sdate + datetime.timedelta(days=delta) #diff is in DAYS (PAY ATTENTION!)
        #ee_edate = datetime.datetime.strftime(temp_edate,'%Y-%m-%d')
        ee_edate = min([datetime.datetime.strptime(str(temp_sdate.year)+'-12-31','%Y-%m-%d'),(temp_edate)])
        print (ee_edate)
        temp = (collection.filterDate(ee_sdate,ee_edate) 
            #.filterMetadata('CLOUD_COVER','less_than',np.mean(clouder)+0.01) #remove the cloudy images -> disrupting reduction
            .filterBounds(ee.Geometry.Rectangle(-106.5, 50,-64, 23)).select(['shortwave_radiation'])\
            #.select(['R_p'+spct,'G_p'+spct,'B_p'+spct,'NIR_p'+spct,'SWIR_p'+spct],['R','G','B','NIR','SWIR']))
            .reduce(ee.Reducer.mean()))
            #.select(['R_mean','G_mean','B_mean','NIR_mean','SWIR_mean'],['R','G','B','NIR','SWIR']))

        X.append(temp)
        dateLims.append([temp_sdate,temp_edate])
        temp_sdate = min(temp_edate,  ee_edate+ datetime.timedelta(days=1))
        print (temp_sdate) 
    return ee.ImageCollection(X)

def export_oneimage(img,folder,name,scale,crs):
  task = ee.batch.Export.image(img, name, {
      'driveFolder':folder,
      'driveFileNamePrefix':name,
      'scale':scale,
      'crs':crs
  })
  task.start()
  while task.status()['state'] == 'RUNNING':
    print ('Running...')
    # Perhaps task.cancel() at some point.
    time.sleep(10)
  print ('Done.'), task.status()



# Transforms an Image Collection with 1 band per Image into a single Image with items as bands


def appendBand(current, previous):
    # Rename the band
    previous=ee.Image(previous)
    current = current.select([0])
    # Append it to the result (Note: only return current item on first element/iteration)
    accum = ee.Algorithms.If(ee.Algorithms.IsEqual(previous,None), current, previous.addBands(ee.Image(current)))
    # Return the accumulation
    return accum



# county_region = ee.FeatureCollection('ft:1S4EB6319wWW2sWQDPhDvmSBIVrD3iEmCLYB7nMM')

print('DEBUG')
#return
county_region = ee.FeatureCollection("TIGER/2016/Counties")

locations = pd.read_csv('locations_final')




collection = ee.ImageCollection("NASA/NLDAS/FORA0125_H002")
delta = 3
imgcoll = cloudReduce(collection,'2018-01-01','2019-01-01',delta) #'2019-05-01','2019-12-01', delta = 1

img=imgcoll.iterate(appendBand)
img=ee.Image(img)

for loc1, loc2, lat, lon in locations.values:
    fname = '{}_{}'.format(int(loc1), int(loc2))
    fname = '{}_{}'.format(int(loc1), int(loc2))
    print(fname)

    # offset = 0.11
    scale  = 500
    crs='EPSG:4326'

    # filter for a county
    region = county_region.filterMetadata('STATEFP', 'equals', str(int(loc1)))
    region = ee.FeatureCollection(region).filterMetadata('COUNTYFP', 'equals', str("{0:03}".format(int(loc2))))
    region = ee.Feature(region.first())
    region.getInfo()


    

    while True:
        try:
            export_oneimage(img.clip(region), '2018_SW_3Day', fname, scale, crs) #MET_test_SW  data_NLDAS_SW_Radiation
        except:
            print ('retry')
            time.sleep(10)
            continue
        break


DEBUG
while 2018-01-01 00:00:00
2018-01-04 00:00:00
2018-01-04 00:00:00
while 2018-01-04 00:00:00
2018-01-07 00:00:00
2018-01-07 00:00:00
while 2018-01-07 00:00:00
2018-01-10 00:00:00
2018-01-10 00:00:00
while 2018-01-10 00:00:00
2018-01-13 00:00:00
2018-01-13 00:00:00
while 2018-01-13 00:00:00
2018-01-16 00:00:00
2018-01-16 00:00:00
while 2018-01-16 00:00:00
2018-01-19 00:00:00
2018-01-19 00:00:00
while 2018-01-19 00:00:00
2018-01-22 00:00:00
2018-01-22 00:00:00
while 2018-01-22 00:00:00
2018-01-25 00:00:00
2018-01-25 00:00:00
while 2018-01-25 00:00:00
2018-01-28 00:00:00
2018-01-28 00:00:00
while 2018-01-28 00:00:00
2018-01-31 00:00:00
2018-01-31 00:00:00
while 2018-01-31 00:00:00
2018-02-03 00:00:00
2018-02-03 00:00:00
while 2018-02-03 00:00:00
2018-02-06 00:00:00
2018-02-06 00:00:00
while 2018-02-06 00:00:00
2018-02-09 00:00:00
2018-02-09 00:00:00
while 2018-02-09 00:00:00
2018-02-12 00:00:00
2018-02-12 00:00:00
while 2018-02-12 00:00:00
2018-02-15 00:00:00
2018-02-15 00:00:00
whil

# NLDAS longwave_radiation

In [81]:
import ee
import time
import sys
import numpy as np
import pandas as pd
import itertools
import os
import urllib
import datetime
ee.Initialize()


def cloudReduce(collection,sdate,edate,delta):
    
    temp_sdate = datetime.datetime.strptime(sdate,'%Y-%m-%d')
    real_edate = min([datetime.datetime.now(),datetime.datetime.strptime(edate,'%Y-%m-%d')])
    yearEnd_edate = min([datetime.datetime.now(),datetime.datetime.strptime(edate,'%Y-%m-%d')])
    X = []
    dateLims = []

    while temp_sdate < real_edate: #go through dates -> get images
        print ("while",temp_sdate)
        ee_sdate = datetime.datetime.strftime(temp_sdate,'%Y-%m-%d')
        temp_edate = temp_sdate + datetime.timedelta(days=delta) #diff is in DAYS (PAY ATTENTION!)
        #ee_edate = datetime.datetime.strftime(temp_edate,'%Y-%m-%d')
        ee_edate = min([datetime.datetime.strptime(str(temp_sdate.year)+'-12-31','%Y-%m-%d'),(temp_edate)])
        print (ee_edate)
        temp = (collection.filterDate(ee_sdate,ee_edate) 
            #.filterMetadata('CLOUD_COVER','less_than',np.mean(clouder)+0.01) #remove the cloudy images -> disrupting reduction
            .filterBounds(ee.Geometry.Rectangle(-106.5, 50,-64, 23)).select(['longwave_radiation'])\
            #.select(['R_p'+spct,'G_p'+spct,'B_p'+spct,'NIR_p'+spct,'SWIR_p'+spct],['R','G','B','NIR','SWIR']))
            .reduce(ee.Reducer.mean()))
            #.select(['R_mean','G_mean','B_mean','NIR_mean','SWIR_mean'],['R','G','B','NIR','SWIR']))

        X.append(temp)
        dateLims.append([temp_sdate,temp_edate])
        temp_sdate = min(temp_edate,  ee_edate+ datetime.timedelta(days=1))
        print (temp_sdate) 
    return ee.ImageCollection(X)

def export_oneimage(img,folder,name,scale,crs):
  task = ee.batch.Export.image(img, name, {
      'driveFolder':folder,
      'driveFileNamePrefix':name,
      'scale':scale,
      'crs':crs
  })
  task.start()
  while task.status()['state'] == 'RUNNING':
    print ('Running...')
    # Perhaps task.cancel() at some point.
    time.sleep(10)
  print ('Done.'), task.status()




#locations = pd.read_csv('locations_final_Selected.csv')


# Transforms an Image Collection with 1 band per Image into a single Image with items as bands
# Author: Jamie Vleeshouwer

def appendBand(current, previous):
    # Rename the band
    previous=ee.Image(previous)
    current = current.select([0])
    # Append it to the result (Note: only return current item on first element/iteration)
    accum = ee.Algorithms.If(ee.Algorithms.IsEqual(previous,None), current, previous.addBands(ee.Image(current)))
    # Return the accumulation
    return accum



# county_region = ee.FeatureCollection('ft:1S4EB6319wWW2sWQDPhDvmSBIVrD3iEmCLYB7nMM')

print('DEBUG')
#return
county_region = ee.FeatureCollection("TIGER/2016/Counties")
# print(county_region.getInfo())
# return



#county_region = county_region.map(lambda f: f.set('STATEFP', ee.Number.parse(f.get('STATEFP'))) )


# locations = pd.read_csv('locations_final_Selected.csv') #missing_LW.csv
#locations = pd.read_csv('missing_LW.csv')
#locations = pd.read_csv('locations_final_MN_MO.csv')
# locations = pd.read_csv('missing_LW.csv')
# locations = pd.read_csv('locations_final_ND_SD_IN.csv')
locations = pd.read_csv('locations_final_Selected.csv') #Location_test.csv locations_final_Selected.csv 

#return
#var image = ee.Image().float().paint(dataset, 'STATEFP');

collection = ee.ImageCollection("NASA/NLDAS/FORA0125_H002")
delta = 3
imgcoll = cloudReduce(collection,'2020-01-01','2021-01-01',delta)

img=imgcoll.iterate(appendBand)
img=ee.Image(img)

for loc1, loc2, lat, lon in locations.values:
    fname = '{}_{}'.format(int(loc1), int(loc2))
    fname = '{}_{}'.format(int(loc1), int(loc2))
    print(fname)

    # offset = 0.11
    scale  = 500
    crs='EPSG:4326'

    # filter for a county
    region = county_region.filterMetadata('STATEFP', 'equals', str(int(loc1)))
    region = ee.FeatureCollection(region).filterMetadata('COUNTYFP', 'equals', str("{0:03}".format(int(loc2))))
    region = ee.Feature(region.first())
    region.getInfo()

    # region = str([
    #     [lat - offset, lon + offset],
    #     [lat + offset, lon + offset],
    #     [lat + offset, lon - offset],
    #     [lat - offset, lon - offset]])
    

    while True:
        try:
            export_oneimage(img.clip(region), '2020_LW_3Day', fname, scale, crs)
        except:
            print ('retry')
            time.sleep(10)
            continue
        break


DEBUG
while 2019-05-01 00:00:00
2019-05-02 00:00:00
2019-05-02 00:00:00
while 2019-05-02 00:00:00
2019-05-03 00:00:00
2019-05-03 00:00:00
while 2019-05-03 00:00:00
2019-05-04 00:00:00
2019-05-04 00:00:00
while 2019-05-04 00:00:00
2019-05-05 00:00:00
2019-05-05 00:00:00
while 2019-05-05 00:00:00
2019-05-06 00:00:00
2019-05-06 00:00:00
while 2019-05-06 00:00:00
2019-05-07 00:00:00
2019-05-07 00:00:00
while 2019-05-07 00:00:00
2019-05-08 00:00:00
2019-05-08 00:00:00
while 2019-05-08 00:00:00
2019-05-09 00:00:00
2019-05-09 00:00:00
while 2019-05-09 00:00:00
2019-05-10 00:00:00
2019-05-10 00:00:00
while 2019-05-10 00:00:00
2019-05-11 00:00:00
2019-05-11 00:00:00
while 2019-05-11 00:00:00
2019-05-12 00:00:00
2019-05-12 00:00:00
while 2019-05-12 00:00:00
2019-05-13 00:00:00
2019-05-13 00:00:00
while 2019-05-13 00:00:00
2019-05-14 00:00:00
2019-05-14 00:00:00
while 2019-05-14 00:00:00
2019-05-15 00:00:00
2019-05-15 00:00:00
while 2019-05-15 00:00:00
2019-05-16 00:00:00
2019-05-16 00:00:00
whil

Done.
17_107
Done.
17_115
Done.
17_123
Done.
17_125
Done.
17_113
Done.
17_129
Done.
17_143
Done.
17_175
Done.
17_179
Done.
17_203
Done.
17_19
Done.
17_53
Done.
17_75
Done.
17_91
Done.
17_105
Done.
17_147
Done.
17_183
Done.
17_23
Done.
17_25
Done.
17_29
Done.
17_33
Done.
17_35
Done.
17_41
Done.
17_45
Done.
17_49
Done.
17_51
Done.
17_79
Done.
17_101
Done.
17_121
Done.
17_139
Done.
17_159
Done.
17_173
Done.
17_7
Done.
17_37
Done.
17_63
Done.
17_89
Done.
17_99
Done.
17_97
Done.
17_111
Done.
17_197
Done.
17_11
Done.
17_15
Done.
17_73
Done.
17_85
Done.
17_103
Done.
17_131
Done.
17_141
Done.
17_155
Done.
17_161
Done.
17_177
Done.
17_195
Done.
17_201
Done.
17_55
Done.
17_59
Done.
17_69
Done.
17_81
Done.
17_127
Done.
17_151
Done.
17_185
Done.
17_191
Done.
17_193
Done.
17_27
Done.
17_87
Done.
17_133
Done.
17_145
Done.
17_153
Done.
17_157
Done.
17_163
Done.
17_189
Done.
17_199
Done.
17_1
Done.
17_9
Done.
17_57
Done.
17_67
Done.
17_71
Done.
17_95
Done.
17_109
Done.
17_169
Done.
17_187
Done.
17_5
D